# Ingest data with Athena
This notebook demonstrates how to set up a database with Athena and query data with it. We are going to use the data we load into S3 in the previous notebook [011_Ingest_tabular_data.ipynb](data_ingestion/011_Ingest_tabular_data_v1.ipynb).

Amazon Athena is a serverless interactive query service that makes it easy to analyze your S3 data with standard SQL. It uses S3 as its underlying data store, and uses Presto with ANSI SQL support, and works with a variety of standard data formats, including CSV, JSON, ORC, Avro, and Parquet. Athena is ideal for quick, ad-hoc querying but it can also handle complex analysis, including large joins, window functions, and arrays. 

To get started, you can point to your data in Amazon S3, define the schema, and start querying using the built-in query editor. Amazon Athena allows you to tap into all your data in S3 without the need to set up complex processes to extract, transform, and load the data (ETL).


## Set up Athena
First, we are going to make sure we have the necessary policies attached to the role that we used to create this notebook to access Athena. You can do this through an IAM client as shown below, or through the AWS console. 

**Note: You would need IAMFullAccess to attach policies to the role.**

#### Attach IAMFullAccess Policy from Console

**1.** Go to **SageMaker Console**, choose **Notebook instances** in the navigation panel, then select your notebook instance to view the details. Then under **Permissions and Encryption**, click on the **IAM role ARN** link and it will take you to your role summary in the **IAM Console**.  

<div>
<img src="image/athena-iam-1.png" width="300"/>
</div>

**2.**  Click on **Create Policy** under **Permissions**.

<div>
<img src="image/athena-iam-2.PNG" width="300"/>
</div>

**3.**  In the **Attach Permissions** page, search for **IAMFullAccess**. It will show up in the policy search results if it has not been attached to your role yet. Select the checkbox for the **IAMFullAccess** Policy, then click **Attach Policy**. You now have the policy successfully attached to your role.


<div>
<img src="image/athena-iam-3.PNG" width="500"/>
</div>

In [1]:
import io
import boto3
import sagemaker
import json
from sagemaker import get_execution_role
import os
import sys

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket() #replace with your own bucket name if you have one
iam = boto3.client('iam')
role = sagemaker.get_execution_role()
role_name = role.split('/')[-1]

When you run the following commend, you will see an error that you cannot list policies if `IAMFullAccess` policy is not attached to your role. Please follow the steps above to attach the IAMFullAccess policy to your role if you see an error.

In [2]:
#check if IAM policy is attached
existing_policies = iam.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']

if 'IAMFullAccess' not in [po['PolicyName'] for po in existing_policies]:
    print('ERROR: You need to attach the IAMFullAccess policy in order to attach policy to the role')
else:
    print('IAMFullAccessPolicy Already Attached')

IAMFullAccessPolicy Already Attached


### Create Policy Document
We will create policies we used to access S3 and Athena. The two policies we will create here are: 
* S3FullAccess: `arn:aws:iam::aws:policy/AmazonS3FullAccess`
* AthenaFullAccess: `arn:aws:iam::aws:policy/AmazonAthenaFullAccess`

You can check the policy document in the IAM console and copy the policy file here.

In [7]:
athena_access_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "athena:*"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "glue:CreateDatabase",
                "glue:DeleteDatabase",
                "glue:GetDatabase",
                "glue:GetDatabases",
                "glue:UpdateDatabase",
                "glue:CreateTable",
                "glue:DeleteTable",
                "glue:BatchDeleteTable",
                "glue:UpdateTable",
                "glue:GetTable",
                "glue:GetTables",
                "glue:BatchCreatePartition",
                "glue:CreatePartition",
                "glue:DeletePartition",
                "glue:BatchDeletePartition",
                "glue:UpdatePartition",
                "glue:GetPartition",
                "glue:GetPartitions",
                "glue:BatchGetPartition"
            ],
            "Resource": [
                "*"
            ]
        },
       
        {
            "Effect": "Allow",
            "Action": [
                "lakeformation:GetDataAccess"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}

In [8]:
#create IAM client
iam = boto3.client('iam')
#create a policy
response = iam.create_policy(
  PolicyName='myAthenaPolicy',
  PolicyDocument=json.dumps(athena_access_role_policy_doc)
)

In [9]:
#get policy ARN
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
policy_athena_arn = f'arn:aws:iam::{account_id}:policy/myAthenaPolicy'

### Attach Policy to Role

In [11]:
# Attach a role policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_athena_arn,
        RoleName= role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached to your role.")
    else:
        print("Unexpected error: %s" % e)

## Intro to PyAthena 

We are going to leverage [PyAthena](https://pypi.org/project/PyAthena/) to connect and run Athena queries. PyAthena is a Python DB API 2.0 (PEP 249) compliant client for Amazon Athena. **Note that you will need to specify the region in which you created the database/table in Athena, making sure your catalog in the specified region that contains the database.** 

In [2]:
!{sys.executable} -m pip install -qU PyAthena==1.10.7

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
from pyathena.util import as_pandas

In [4]:
# Set Athena database name
database_name = 'tabularbh'

In [5]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = 's3://{0}/athena/staging'.format(bucket)

In [6]:
# write the SQL statement to execute
statement = 'CREATE DATABASE IF NOT EXISTS {}'.format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS tabularbh


In [7]:
# connect to s3 using PyAthena
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

### Register Table with Athena 
When you run a CREATE TABLE query in Athena, you register your table with the AWS Glue Data Catalog. 

To specify the path to your data in Amazon S3, use the LOCATION property, as shown in the following example: `LOCATION s3://bucketname/folder/`

The LOCATION in Amazon S3 specifies all of the files representing your table. Athena reads all data stored in `s3://bucketname/folder/`. If you have data that you do not want Athena to read, do not store that data in the same Amazon S3 folder as the data you want Athena to read. If you are leveraging partitioning, to ensure Athena scans data within a partition, your WHERE filter must include the partition. For more information, see [Table Location and Partitions](https://docs.aws.amazon.com/athena/latest/ug/tables-location-format.html).

In [8]:
prefix = 'data/tabular'
filename_key = 'boston_house'

In [9]:
data_s3_location = "s3://{}/{}/{}/".format(bucket, prefix, filename_key)

In [10]:
table_name_csv = 'boston_house_athena'

In [14]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        CRIM double,
        ZN double,
        INDUS double,
        CHAS double,
        NOX double,
        RM double,
        AGE double,
        DIS double, 
        RAD double, 
        TAX double,
        PTRATIO double, 
        B double, 
        LSTAT double,
        target double

) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(database_name, table_name_csv, data_s3_location)

In [15]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [16]:
#verify the table has been created
statement = 'SHOW TABLES in {}'.format(database_name)
cursor.execute(statement)

df_show = as_pandas(cursor)
df_show.head(5)

,tab_name
0,boston_house
1,boston_house_athena


In [17]:
#run a sample query
statement = """SELECT * FROM {}.{}
LIMIT 100""".format(database_name, table_name_csv)
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [18]:
df = as_pandas(cursor)
df.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Alternatives: Use AWS Data Wrangler to query data

In [19]:
!{sys.executable} -m pip install -qU awswrangler==1.2.0

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [20]:
import awswrangler as wr

#### Glue Catalog

In [21]:
for table in wr.catalog.get_tables(database=database_name):
    print(table['Name'])

boston_house
boston_house_athena


#### Athena

In [22]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 100'.format(table_name_csv),
    database=database_name
)

CPU times: user 265 ms, sys: 15.7 ms, total: 281 ms
Wall time: 5.05 s


In [23]:
df.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Citation
Boston Housing data,  Harrison, D. and Rubinfeld, D.L. `Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.

Data Science On AWS workshops, Chris Fregly, Antje Barth, https://www.datascienceonaws.com/